# Extended Events

***You will need to have followed the steps in the 00-CreateContainers notebook to use this notebook***

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

dbatools is **awesome** with Extended Events

First we will set up the variables and connections to the containers

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

 Creating connection to the containers
We have a connection to the containers


## Listing Extended Events

Which Extended Event sessions do we have on our instances ?

This will give you 

- The name of the session
- If it is running
- When it started
- If it will start on server startup (Autostart = True)
- The targets
- The target filename
- The events

In [2]:
Get-DbaXESession -SqlInstance $SQL1


ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 0a227436a4b8
Name         : 15 Second IO Error
Status       : Running
StartTime    : 10/03/2020 10:38:02
AutoStart    : True
State        : Existing
Targets      : {package0.event_file}
TargetFile   : {/var/opt/mssql/log\15_second_io_error}
Events       : {sqlserver.file_read_completed, sqlserver.file_write_completed}
MaxMemory    : 4096
MaxEventSize : 0

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 0a227436a4b8
Name         : AlwaysOn_health
Status       : Stopped
StartTime    : 
AutoStart    : False
State        : Existing
Targets      : {package0.event_file}
TargetFile   : {/var/opt/mssql/log\AlwaysOn_health.xel}
Events       : {sqlserver.alwayson_ddl_executed, sqlserver.availability_group_lease_expired, 
               sqlserver.availability_replica_automatic_failover_validation, 
               sqlserver.availability_replica_manager_state_change…}
MaxMemory    : 4096
MaxEventSize : 0

Compute

## Starting and stopping Extended Events Sessions

It is no surprise that you can start and stop extended events sessions easily with dbatools. We like to make command names easy to remember!

`Start-DbaXeSession`  
`Stop-DbaXeSession`

The following session is running

In [3]:
Get-DbaXESession -SqlInstance $SQL1 -Session 'Blocked Process Report'


ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 0a227436a4b8
Name         : Blocked Process Report
Status       : Running
StartTime    : 10/03/2020 10:38:02
AutoStart    : True
State        : Existing
Targets      : {package0.event_file}
TargetFile   : {/var/opt/mssql/log\Blocked Process Report}
Events       : {sqlserver.blocked_process_report}
MaxMemory    : 8192
MaxEventSize : 0




We can stop it


In [4]:
Stop-DbaXESession  -SqlInstance $SQL1 -Session 'Blocked Process Report'


ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 0a227436a4b8
Name         : Blocked Process Report
Status       : Stopped
StartTime    : 
AutoStart    : True
State        : Existing
Targets      : {package0.event_file}
TargetFile   : {/var/opt/mssql/log\Blocked Process Report}
Events       : {sqlserver.blocked_process_report}
MaxMemory    : 8192
MaxEventSize : 0




and start it again


In [5]:
Start-DbaXESession  -SqlInstance $SQL1 -Session 'Blocked Process Report'


ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 0a227436a4b8
Name         : Blocked Process Report
Status       : Running
StartTime    : 10/03/2020 10:38:29
AutoStart    : True
State        : Existing
Targets      : {package0.event_file}
TargetFile   : {/var/opt/mssql/log\Blocked Process Report}
Events       : {sqlserver.blocked_process_report}
MaxMemory    : 8192
MaxEventSize : 0




We can watch the live data - Lets create a different process to watch the Blocked Process Session.

This code will open a Windows PowerShell session, right click to paste the code and it will prompt for a password which is `dbatools.IO`
Leave that running and move to the next step

In [7]:
## When PowerShell opens - right click to paste the command
#  
@"
## the password is dbatools.IO
Get-DbaXESession -SqlInstance 'localhost,15592' -SqlCredential sqladmin -Session 'Blocked Process Report' | Watch-DbaXESession  
"@ | clip

# 
Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

This code will create some blocked processes, once you have run it you can look at the Windows PowerShell window and see the details, close the window when you are done!

In [11]:
$query1 = "
BEGIN TRANSACTION
  SELECT * FROM [dbo].[authors] WITH (TABLOCKX, HOLDLOCK)
    WAITFOR DELAY '00:00:30' -- 30 seconds
ROLLBACK TRANSACTION
"
$query2 = "SELECT TOP 1 * FROM [dbo].[authors]" 
$Queries = $query1, $query2, $query2 , $query2, $query2, $query2, $query2, $query2, $query2, $query2
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$Queries |ForEach-Object -Parallel  {
$Credential = Import-Clixml -Path $Using:FolderPath\sqladmin.cred
Invoke-DbaQuery -SqlInstance 'localhost,15592' -SqlCredential $Credential -Database pubs -Query $psitem 
}


au_id    : 172-32-1176
au_lname : White
au_fname : Johnson
phone    : 408 496-7223
address  : 10932 Bigge Rd.
city     : Menlo Park
state    : CA
zip      : 94025
contract : True

au_id    : 213-46-8915
au_lname : Green
au_fname : Marjorie
phone    : 415 986-7020
address  : 309 63rd St. #411
city     : Oakland
state    : CA
zip      : 94618
contract : True

au_id    : 238-95-7766
au_lname : Carson
au_fname : Cheryl
phone    : 415 548-7723
address  : 589 Darwin Ln.
city     : Berkeley
state    : CA
zip      : 94705
contract : True

au_id    : 172-32-1176
au_lname : White
au_fname : Johnson
phone    : 408 496-7223
address  : 10932 Bigge Rd.
city     : Menlo Park
state    : CA
zip      : 94025
contract : True

au_id    : 172-32-1176
au_lname : White
au_fname : Johnson
phone    : 408 496-7223
address  : 10932 Bigge Rd.
city     : Menlo Park
state    : CA
zip      : 94025
contract : True

au_id    : 267-41-2394
au_lname : O'Leary
au_fname : Michael
phone    : 408 286-2428
address  : 22 Cle

Thats really cool, you can see the data flying by on the screen  
  
![dbatools](.\images\blockedprocessXE.png )  

But what about if you would like to read the data

There is no need to learn the XML shredding :-)

We can read the file.

First we need to know where the file is

In [13]:
Get-DbaXESessionTarget -SqlInstance $SQL1 -Session 'Blocked Process Report'


ComputerName  : localhost
InstanceName  : MSSQLSERVER
SqlInstance   : 0a227436a4b8
Session       : Blocked Process Report
SessionStatus : Running
Name          : package0.event_file
ID            : 2
Field         : {filename, increment, is_indexed_file_target, lazy_create_blob…}
PackageName   : package0
File          : {/var/opt/mssql/log\Blocked Process Report}
Description   : Use the event_file target to save the event data to an XEL file, which can be 
                archived and used for later analysis and review. You can merge multiple XEL files 
                to view the combined data from separate event sessions.
ScriptName    : package0.event_file




If we weren't using containers you could use `Get-DbaXeSessionTargetFile` and if you were on Windows and had access to the admin UNC share, you could do

`Get-DbaXESession -SqlInstance $SQL1 -Session 'Blocked Process Report' | Read-DbaXEFile`

But we arent so lucky. Lets stop the session and copy the files to the directory we have mounted for our containers (I copy all the files because after 5 minutes of trying to only copy teh xel files with bash I gave up)

In [42]:
docker exec -i dbatools_SQL2019_1 cp -R /var/opt/mssql/log/ /var/opt/mssql/backups/

With the files moved to a place we can access them, we can read the files with one line of code. It won't work in the notebook but the code below will open a Windows PowerShell session and the you can right click to paste the command and read the data. `CTRL + C ` will stop it and you can just close the PowerShell session when it is done

In [53]:
## When PowerShell opens - right click to paste the command
#     
"Get-ChildItem $FolderPath\log\Block* | Read-DbaXEFile" | clip

Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

But we are using PowerShell which means we have all of the tools available to us :-)

How about if we want the XE event data in a CSV file ?

Again we need to open a PowerShell session and right click and when it is done, close it and come back here

In [ ]:
## When PowerShell opens - right click to paste the command
#     
"Get-ChildItem $FolderPath\log\Block* | Read-DbaXEFile | Export-Csv -Path $FolderPath\BlockedProcess.csv -NoClobber" | clip

Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

Now we can read the CSV file


In [2]:
$BlockedProcesses = Import-Csv $FolderPath\BlockedProcess.csv 
$BlockedProcesses | Select -First 1


name                : blocked_process_report
timestamp           : 10/03/2020 10:39:16 +00:00
blocked_process     : <blocked-process-report monitorLoop="15"><blocked-process><process 
                      id="process11f6a6c108" taskpriority="0" logused="0" waitresource="OBJECT: 
                      8:581577110:0 " waittime="5536" ownerId="7549" transactionname="SELECT" 
                      lasttranstarted="2020-03-10T10:39:11.277" XDES="0x11f7925be8" lockMode="IS" 
                      schedulerid="3" kpid="748" status="suspended" spid="61" sbid="0" ecid="0" 
                      priority="0" trancount="0" lastbatchstarted="2020-03-10T10:39:11.273" 
                      lastbatchcompleted="2020-03-10T10:39:11.277" 
                      lastattention="2020-03-10T10:39:11.200" clientapp="dbatools PowerShell 
                      module - dbatools.io" hostname="BEARDXPS" hostpid="5212" 
                      loginname="sqladmin" isolationlevel="read committed (2)" xactid="7549"

We could even put them in a database (Excel File with ImportExcel, Email, Word Document, back to XML, into Azure etc etc)

In [ ]:
$BlockedProcesses | Write-DbaDataTable -SqlInstance $SQL1 -Database tempdb -Table BlockedProcess -AutoCreateTable 

More to come here

# Clean Up

The 99-CleanUpContainers notebook will remove the containers, files and directory - it will leave the image so you do not have to download it again!